In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

import rnn
import train
import brainfuck

In [2]:
embedding_size = 10
hidden_size = 35
output_size = len(rnn.char.keys())
n_layers = 2
batch_size = 64
GAMMA = 0.99 # for exponential moving avarage constant from paper




In [3]:
print(rnn.token_num)

9


In [4]:
model = rnn.BFgen(rnn.token_num, embedding_size, hidden_size, output_size, n_layers, batch_size)

In [5]:
print(train.objective_PG(model, lambda x : np.array([1] * model.batch_size)), model.entropy)

(Variable containing:
-133.2986
[torch.FloatTensor of size 1]
, Variable containing:
 13287.1973
[torch.FloatTensor of size 1]
)


In [6]:
print(train.objective_PQT(model))

Variable containing:
-208.4926
[torch.FloatTensor of size 1]



In [7]:
print(model.pqt_programs)

['.][--<][.>>.[][+]>,,-->[.+>.<->>>+>+-]][][[,..[->,+]+]<]]][]<>[-,-,,-++..><<]>.-]+-[<,>-[]>,[+.+><[<'
 '-+>,][<<<+<[[],]+,.<<>-++,>,.,,[.,+,-<<++<>.[>>[>]+]<[--]>-]+++],[>--]]<><[<>]-.]>[[>-..-,<+<]<<>[[+'
 '>+->+[.+>,<.<[+[]<.>,.[..[.[,[.,[.<]>,.]-],]<[<.[>>-[,,<+.+>[+<.[[>><[],]-].],[,--[-+],<,><]>+[>++-['
 '<-[><<<+,+[+.>++,+->+.<]+[+.[--[,<.--.-,++]<.<++-<]],.-<[->[.>>,-[].><[+<,-[>[.]-]+<.],>.]>.>+>-[].+'
 '>>,][>>]]..,[-,<-.,[..[>.+>.[],><<,->,[.,]][.,[+>,,-[[>>-[-<<-,,>]<<]->].+[>>--[>-]>+.<]<[.[+-][<]-['
 '--->.+..<<<,,<<<<+,-,[->.]+-+++><-<++]],-.>+.,,>.,[[,]>[-,.>]<].[.++<][.-[-]<].<,+[+,-]<.<>,]<>>.-<<'
 ',>.,],][[[+]][..+-[>+-..>[,-.[,[>-[>.[-]-.,[.[<>-[>.-[.][][-<+->[[<[]>->+-,.>-..[,->]]>-[>,.[>+-[>]-'
 '.+-]>,++[][,.,<,++>,],-.<,,<->,+.+]<>,],>[+,[-<,.-.-[[+->+,>,[]>><.>,]>>-<-[,,[,>[..<[->.<.->]>><-++'
 '<<]<>[+<,<.>].>+[].,,>..]+,<<<.]..[[+<+[,-+]>+--.[--]+.][]]],].-><.+++<,].-><+-],>]+>><<-.->><-][-]]'
 '[<+<,+,>>,<,,++<.+..<<,>,[<,,>-+,-,[+[+<+-,.-+[.],<-<-.+-.[.]<

In [8]:
bfAdd = ",>++++++[<-------->-],[<+>-]<++++++++."
bfAdd2 = ",>++++++[<-------->-],[<+>-]<."
bfAdd3 = ",>++++++[<-------->-],[<+>-]<++..."
bfAdd4 = ",>++++++[<-------->-],[<+>-]<+++............"
addBatch = [bfAdd, bfAdd2, bfAdd3, bfAdd4]
bfAddIn = ['11', '12', '23']
bfAddOut = ['2','3','5']

In [9]:
reward = train.batch_reward(bfAddIn, bfAddOut)

In [10]:
reward(addBatch)

array([  76.5,   76.8,  -77.1, -768.3])

In [11]:
reverse_in = ["a", "ab", "abc", "abcd", "abcde"]
reverse_out = ["a", "ba", "cba", "dcba", "edcba"]
reverse_reward = train.batch_reward(reverse_in, reverse_out, scaling_factor=1./3840.)

In [12]:
model.clear_pq()
train.train_pqt_pg(model, reverse_reward, NPE=1000000)

Epoch Obj  Sample
----- ----- ------
Variable containing:
 1540.5519
[torch.FloatTensor of size 1]

Variable containing:
-208.0899
[torch.FloatTensor of size 1]

Variable containing:
 207.6124
[torch.FloatTensor of size 1]

-10.451369897460937
pqt [('[[<<<<.<+--]>][.-]]+,,]-.<,<>]<]+]+<<.]>,[<[<>.,..[<+<-,+-+><+->>.+-][+>,[-[>>-+,.[.,>]-,<.-]-.<++-,', -2.0703125), ('<><,[-++-[-[>+[.[-+.<+[----.-[[.>>[,>-,.,+><[]<,<>,++-+<<>++,,>><>],,-.+.,.[+.>.-+][+][<-],+,[.-,,<<', -0.13723958333333333), ('<+>[>>-[+,>-[<]][+--]-<,-]-]<,[[,]].+.[[[-<[>.[-..[,]]--[<+-]..+>-,<>,<<-+<+[[.],,+<>+.[<-..-]]<-]<,', 0.0), ('.>-.<[+.+,.>.>[]>>>+,[-,.<]-.<,-,][-,,>+.[>]<..-<].++++,-->--<>-<<.>+>>+,,><[+>.+[-][<>[]].<>-..-++.', 0.0), ('<]>.]-]..[,[-,++><-,-<,-.<>.>>>>.-+-.+.,+<>->><,,+>.+>-+>.[-,,+]+[.[+,[<[>[+<[<,-]->.-+>>>>.--+-<[[,', 0.0), ('>-,]<.]-[,<<[.>->[-+[.-[<-]>.<-<-+][>-[+].+>>-]+[->[>[>>,-]<<,,.-[<]>]>->-]].[..[]<+.[>[+,-]>,-[-+[<', 0.0), ('-]..-[>],+]],>++->+<+.,.-,-<.,.+.--.<].+],+]]->+-[,--[,<-][

KeyboardInterrupt: 

In [15]:
import bfCompiler
brainfuck.BF("-+[,-><<<.+.,+<-,><+-..]-<..+.>>[[<[-<+-.]-]+>+.+>-.]-,+>]][>]+]..><->][,[>,..<,<[[]][[++<>-],+..+--", "abc")

'\xff\xff'

In [16]:
expected_code = "><.><"
program_batch_code = [".....", "..<..", "[][][][][[][[][][][][][][]]]", "]"]

In [17]:
simple_rew = train.simplified_batch_reward(expected_code)

In [18]:
simple_rew(program_batch_code)

array([ 0.996875  ,  0.99609375, -1.        ,  0.19921875])

In [19]:
training_expected_code = ">>>.." * 20
print len(training_expected_code),training_expected_code

100 >>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..


In [20]:
training_rew = train.simplified_batch_reward(training_expected_code)
training_rew(program_batch_code)

array([0.04988281, 0.04988281, 0.27890625, 0.00996094])

In [21]:
def ultra_simple_rew(expected_code):
    def hamming_distance(seq1, seq2):
        assert len(seq1) == len(seq2)
        return sum(elem1 != elem2 for elem1, elem2 in zip(seq1, seq2))
    
    code_len = float(len(expected_code))
    def reward_code(generated_code_batch):
        rewards = np.array(map(lambda x: 1 - 2*(hamming_distance(x, expected_code) / code_len), generated_code_batch))
        return rewards
    return reward_code

In [22]:
simple_rew = ultra_simple_rew(training_expected_code)
sample1 = ">" * 100
sample2 = "." * 100
sample3 = "[]" * 50
sample4 = "[]><+-.,[]"  * 10
samples = [sample1, sample2, sample3, sample4]

In [23]:
simple_rew(samples)

array([ 0.2, -0.2, -1. , -0.8])

In [24]:
model.clear_pq()
model.init_weights()
train.train_pqt_pg(model, simple_rew, NPE=1000000)

Epoch Obj  Sample
----- ----- ------
Variable containing:
-1419.2838
[torch.FloatTensor of size 1]

Variable containing:
-208.7697
[torch.FloatTensor of size 1]

Variable containing:
 206.2663
[torch.FloatTensor of size 1]

-0.864852556352991
pqt [(']+].]]-<+[.->++>--,-,-<<-[]+.<+-+<+<<..>,>[-.,[-,,<<>[[-.<..-[+[[+>-,[+>>-.-.+[<,-..[+-]-,.<+]][].-,', -0.7), ('+[,-><>>.]++]+.].>[[..><,-<<[>.<],+<>>[,>]]]<>,-]<,>>---+<>,>]--],[,<<[+--.[,]<--[]<>[->[]+[[-<[[<,,', -0.7), ('-[-.]+.+->[,--+]->.,...<+<->+.>->]<+>]-[.+.<,[]..>+]+][<-<-]]<<+<-[+,--[].-[],>.<>++[>]>,+-<,+-+,[[.', -0.7), ('<>-,.[<.>,.><[+],]--<<-.-]+]><][,,+.-.+<.+-.>]>,..+>].--+]+-,[<>[[--.--++,....,,-+<+[,].,..+-][]+<..', -0.7), ('>+,<]+,-<,>[[>++<-+.<,,.,.].-<+.[.+.-[,>.,][.+[[.]>>[,[,[][-]+,..>[<-[+][,-[<[..<[]<]+]-[]<>[,-+,+<-', -0.7), ('..]>].+][-]+,[+-,]..>,<.<.->-][+,[>,[+>[-]<..<-.[]+,-][[<[>.>.<]-<<]-]+[][.>,<.->>>[][.>,[>..+<[[-<]', -0.6599999999999999), ('><,[]>.+>,[-+-[,],.<>.-[.+]+.]][[].+[-.<]>>,+>..+>,.+,[<<<.,+]-

KeyboardInterrupt: 